# Presentazione esperimento

**Panoramica**
Estende la copertura alle domande Medium introducendo conoscenza su pianeti, ristoranti e licenze in tutta la pipeline.

**Pipeline dati**
- Parsing/classificazione/estrazione identici al notebook precedente ma con output arricchito.
- Mapping dedicati (`create_mappings_planets_restaurant_skills`, `create_mappings_technique_ingredient`) per collegare piatti a pianeti, locali e skill degli chef.
- Valutazione `evaluate_questions` su Easy+Medium per avere una singola metrica.

**Evoluzione dell'engine**
- `engine_medium` viene usato al completo: ingredienti, tecniche, pianeti, ristoranti e vincoli di licenza convivono nello stesso agente.
- Le set-ops (`intersect`, `subtract`, `union`) consentono di comporre query multi-condizione che miscelano geografia, regolamenti e ingredienti.

**Ruolo nel percorso**
Rappresenta il primo esperimento truly multi-sorgente e costituisce la base su cui innestare le fonti normative e spaziali dell'esperimento Hard.

**Performance (Jaccard)**: 99.6%



# Setup

In [ ]:
import os
from pathlib import Path
import sys


	
cwd = Path.cwd().resolve()
project_dir = cwd.parent.parent

if str(project_dir) not in sys.path:
	sys.path.insert(0, str(project_dir))
	

dataset_file_path = project_dir / "Dataset"
artifacts_file_path = cwd / "artifacts"

if not os.path.exists(artifacts_file_path):
        os.makedirs(artifacts_file_path)

# Preprocessing

## Parsing e aggregazione

In [ ]:
from src.preprocessing.menu_ingestion import group_and_concatenate_documents, parse_documents_in_directory
from src.utils import write_json


menus_path = dataset_file_path / "Knowledge_base" / "menu"
documents_pages = parse_documents_in_directory(document_path=menus_path)
documents = group_and_concatenate_documents(documents=documents_pages)
write_json(documents, artifacts_file_path / "parsed_menus.json")

for doc_name, doc_text in list(documents.items())[:5]:
    print(f"Document: {doc_name}\nContent Preview: {doc_text[:100]}...\n")

## Classificazione

In [ ]:
from src.preprocessing.menu_classification import classify_menu

classifications = classify_menu(text_extracted=documents, model_name="gpt-4.1")
write_json(classifications, artifacts_file_path / "menu_classifications.json")

for doc_name, classification in list(classifications.items())[:5]:
    print(f"Document: {doc_name}\nClassification: {classification}\n")

## Estrazione strutturata

In [ ]:
from src.preprocessing.menu_extraction import extract_info_from_menus
from src.utils import read_json, write_json

extracted_info = extract_info_from_menus(documents=documents, classifications=classifications, model_name="gpt-4.1")
write_json(extracted_info, artifacts_file_path / "extracted_menu_info.json")

for info in extracted_info[:5]:
    restaurant_name = info.get("restaurant_name", "Unknown")
    print(f"Document: {restaurant_name}\nExtracted Info: {info}\n")

## Creazione di mapping

In [ ]:
from src.preprocessing.menu_mapping import create_mappings_planets_restaurant_skills
from src.utils import read_json,write_json


dish_mapping = read_json(dataset_file_path / "ground_truth" / "dish_mapping.json")

planet_to_dishes, restaurant_to_dishes, skill_to_dishes = create_mappings_planets_restaurant_skills(extracted_info=extracted_info, 
                                                                                                    dish_mapping=dish_mapping)

write_json(planet_to_dishes, artifacts_file_path / 'planet_to_dishes.json')
write_json(restaurant_to_dishes, artifacts_file_path / 'restaurant_to_dishes.json')
write_json(skill_to_dishes, artifacts_file_path / 'skill_to_dishes.json')


In [ ]:
from src.preprocessing.menu_mapping import create_mappings_technique_ingredient

ingredient_to_dishes, technique_to_dishes = create_mappings_technique_ingredient(extracted_info=extracted_info, 
                                                                                 dish_mapping=dish_mapping)

write_json(ingredient_to_dishes, artifacts_file_path / 'ingredient_to_dishes.json')
write_json(technique_to_dishes, artifacts_file_path / 'technique_to_dishes.json')

# Engine

## Agente conversazionale / interrogazioni

In [ ]:
from src.ai.agents.engine_medium import get_agent, query_dish_ids

agent = get_agent(model_name="grok-4-1-fast-reasoning")
response = query_dish_ids(question="Quali piatti sono stati creati dallo chef con almeno una licenza Luce di grado 3 utilizzando la tecnica della Cottura Sottovuoto Multirealità Collassante?", agent=agent)
print(response)

## Valutazione

In [ ]:
from src.evaluation.questions_evaluation import evaluate_questions


question_path = dataset_file_path / "domande.csv"
ground_truth_path = dataset_file_path / "ground_truth" / "ground_truth_mapped.csv"
eval_df = evaluate_questions(agent=agent, question_path=question_path, ground_truth_path=ground_truth_path, level="medium")
eval_df.to_csv(artifacts_file_path / "medium_questions_evaluation_results.csv", index=False)

In [ ]:
score =  round(eval_df['score'].mean()*100, 2)
print(f"Accuracy: {score}%")